In [ ]:
!pip install pymystem3

In [ ]:
!pip install natasha

In [ ]:
!pip install spacy

In [ ]:
! python -m spacy download ru_core_news_sm

## Домашнее задание номер 2

На последнем семинаре мы проанализировали несколько различных морфологических теггеров. Как же узнать, какой использовать? Давайте сравним их качество!

В этой домашке вам будет нужно найти тексты на русском языке (размер корпуса не менее 200 слов), 
в которых  будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их вручную 
– с помощью этих текстов мы будем оценивать качество работы наших теггеров. В текстах размечаем только части речи, ничего больше!
1. (1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.

Текст в файле **sents.txt**, разметка текста в файле **gold_standard.txt**. 

Трудные для постеггинга моменты в моём тексте:
- аббревиатуры *ООП*, *ВПО*, *ФГОАУ* и т.п. -- их можно не распознать совсем или спутать с другими частями речи, например, при соответствующих окончаниях
- употребительные сокращения *ст. научн. сотр.* и т.п. -- этих сокращений вполне вероятно нет в словарях моделей и, особенно когда за ними скрываются не существительные, это будет трудный случай для моделей
- составной предлог *за счет*, который я решила делить на две части в угоду орфографии, но помечать обе части как предлог.
- конверсия *родные*, *исполняющий* в И. о., *обратное* в *в обратном*, *данные*
- слова через дефис типа *день-другой*, *ну-ка*, *ходить-бродить* - возникает в первую очередь вопрос, как их делить на токены, а затем, соответственной, какую часть речи приписывать (результат будет различным в зависимости от результатов токенизации)
- предложение с переставленными буквами внутри слова -- человек может понимать такие предложения, поэтому хотелось бы, чтобы и модели могли
- омоформы вроде *стекла*, *мечи*, *косой*, *коса*, *кося* и пр., которые, очевидно, можно перепутать
- неочевидные демонстративы *прочий*, *некоторый*
- слова, которых скорее всего нет в словарях у использующихся моделей *моджибаке*, *памимимный*, *ведомостичка* и т.п.
- нелитературные формы слов *побежду*, *убедю* -- это формы глаголов, которых наверняка нет в словарях моделей, а мы кроме того знаем из ридинга, что глаголы модели предсказывают с меньшей вероятностью, чем существительные и т.п.

Для разметки я взяла тегсет, основанный на стандарте UD, это удобно, так как этот стандарт на данный момент считается наиболее универсальными. Тем не менее, я внесла некоторые изменения и упрощения, в целом из неочевидного:
- соответствующие имена собственные считаются существительными (позволит верно соотнести категории из UD и из mystem, который, соответственно, считает имена собственные просто существительными)
- вспомогательные глаголы типа *быть* считаются глаголами (это более характерно для русской традиции и позволит избежать сложностей при работе с тегами, более ориентированными на русский язык как у mystem)
- все виды причастий и деепричастий -- также формы глаголов (это удобно, потому что сокращает количество частеречных тегов, кроме того причастия и деепричастия однозначно образовываются от глаголов и невыделение их в отдельные классы более универсально с типологической точки зрения)
- *прочий*, *некоторый*, *один* (в соотв. контексте) -- детерминативы, так как это логично для стандарта UD и в стандарте mystem тоже находит соответствие -- категорию APRO, объединяющую местоимённые прилагательные. Тогда в категории местоимений остаются только местоимённые существительные.

In [66]:
with open("sents.txt", 'r', encoding='utf-8') as file:
    sents = file.read()

sents = sents.split('\n')
sents[:5]

['Финансовое обеспечение деятельности Федерального государственного автономного образовательного учреждения высшего профессионального образования ФГОАУ ВПО Национального исследовательского университета «Высшая школа экономики» (далее НИУ ВШЭ) за счет средств федерального бюджета осуществляет Управление делами Президента Российской Федерации.',
 'К 2018 году 100% ООП ВПО прошли экспертизу с участием международных экспертов.',
 'Под Марксом, в кресло вкресленный с высоким окладом, высок и гладок, сидит облачённый ответственный.',
 'Каждый на месте: невеста — в тресте, кум — в Гум, брат — в наркомат.',
 'Все шире периферия родных, и в ведомостичках узких не вместишь всех сортов наградных — спецставки, тантьемы, нагрузки!']

Создаём список списков токенов и POS-тегов на основе образцовой разметки.

In [87]:
with open("gold_standard.txt", 'r', encoding='utf-8') as file:
    tokens = file.read()
    
tokens = tokens.split('\n')
gold_standard_list = []
for token in tokens:
    gold_standard_list.append(token.split('\t'))

In [88]:
gold_standard_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

2. (3 балла) Потом вам будет нужно взять три  POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.

### Mystem

In [15]:
from pymystem3 import Mystem

In [130]:
m = Mystem()
mystem_analysis_list = []
for sent in sents:
    sent_analysis = m.analyze(sent)
    for i in range(len(sent_analysis)):
        # Исключаем из анализа пробелы и пунктуацию
        if 'analysis' in sent_analysis[i]:
            lexeme = sent_analysis[i]['text']
            # Вписываем отдельно слова, для которых анализатор не смог определить часть речи
            if sent_analysis[i]['analysis'] != []:
                morph = sent_analysis[i]['analysis'][0]['gr']
                pos = morph.split('=')[0].split(',')[0]
            else:
                pos = None
            mystem_analysis_list.append([lexeme, pos])

In [132]:
mystem_analysis_list[:5]

[['Финансовое', 'A'],
 ['обеспечение', 'S'],
 ['деятельности', 'S'],
 ['Федерального', 'A'],
 ['государственного', 'A']]

### Natasha

In [106]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc

In [107]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [117]:
natasha_analysis = Doc(" ".join(sents))
natasha_analysis.segment(segmenter)
natasha_analysis.tag_morph(morph_tagger)

In [138]:
natasha_analysis.sents[0].morph.tokens[0]

MorphToken(
    text='Финансовое',
    pos='ADJ',
    feats={'Case': 'Nom',
     'Degree': 'Pos',
     'Gender': 'Neut',
     'Number': 'Sing'}
)

### Spacy

In [121]:
import spacy

In [122]:
nlp = spacy.load("ru_core_news_sm")

In [156]:
spacy_doc = nlp(" ".join(sents))
spacy_analysis_list = []
for token in spacy_doc:
    spacy_analysis_list.append([token.text, token.pos_])

In [157]:
spacy_analysis_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

3. (2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи  могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

In [137]:
def convert_tags(word_pos_tag, model):
    if model.lower() == 'mystem':
        dict_corr = {'A': 'ADJ', 'PR': 'ADP', 'S': 'NOUN', 'V': 'VERB', 'ANUM': 'NUM',
                                      'APRO': 'DET', 'ADVPRO': 'ADV', 'SPRO': 'PRO'}
    elif model.lower() == 'natasha' or model.lower() == 'spacy':
        dict_corr = {'PROPN': 'NOUN', 'AUX': 'VERB', 'CCONJ': 'CONJ', 'SCONJ': 'CONJ', 'PUNCT': None}
    
    if word_pos_tag in dict_corr:
        return(dict_corr[word_pos_tag])
    else:
        return(word_pos_tag)

In [158]:
mystem_conv_analysis_list = []
for mystem_word_analysis in mystem_analysis_list:
    mystem_conv_tag = convert_tags(mystem_word_analysis[1], 'mystem')
    mystem_conv_analysis_list.append([mystem_word_analysis[0], mystem_conv_tag])

In [159]:
mystem_conv_analysis_list[:5]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ']]

In [148]:
natasha_conv_analysis_list = []
for natasha_sent_analysis in natasha_analysis.sents:
    for natasha_word_analysis in natasha_sent_analysis.morph.tokens:
        natasha_conv_word_analysis = [natasha_word_analysis.text,
                                           convert_tags(natasha_word_analysis.pos, 'natasha')]
        if natasha_conv_word_analysis[1] != None:
            natasha_conv_analysis_list.append(natasha_conv_word_analysis)

In [155]:
natasha_conv_analysis_list[:5]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ']]

In [160]:
spacy_conv_analysis_list = []
for spacy_word_analysis in spacy_analysis_list:
    spacy_conv_tag = convert_tags(spacy_word_analysis[1], 'spacy')
    if spacy_conv_tag != None:
        spacy_conv_analysis_list.append([spacy_word_analysis[0], spacy_conv_tag])

In [162]:
spacy_conv_analysis_list[:10]

[['Финансовое', 'ADJ'],
 ['обеспечение', 'NOUN'],
 ['деятельности', 'NOUN'],
 ['Федерального', 'ADJ'],
 ['государственного', 'ADJ'],
 ['автономного', 'ADJ'],
 ['образовательного', 'ADJ'],
 ['учреждения', 'NOUN'],
 ['высшего', 'ADJ'],
 ['профессионального', 'ADJ']]

In [178]:
len(gold_standard_list), len(mystem_conv_analysis_list), len(natasha_conv_analysis_list), len(spacy_conv_analysis_list)

(231, 231, 232, 240)

In [177]:
# final_analysis_list = []

#     final_analysis_list.append([gold_standard_list[i][0], gold_standard_list[i][1],
#                                 mystem_conv_analysis_list[i][0], mystem_conv_analysis_list[i][1],
#                                 natasha_conv_analysis_list[i][0], natasha_conv_analysis_list[i][1],
#                                 spacy_conv_analysis_list[i][0], spacy_conv_analysis_list[i][1]])
import csv   
    
with open('analysis_comparison.tsv', 'w', newline='') as file:
    longest_analysis_len = max(len(gold_standard_list), len(mystem_conv_analysis_list),
                               len(natasha_conv_analysis_list), len(spacy_conv_analysis_list))
    for i in range(longest_analysis):
        gold_standard_analysis = gold_standard_list[i]
        mystem_analysis = mystem_conv_analysis_list[i]
        natasha_analysis = natasha_conv_analysis_list[i]
        spacy_analysis = spacy_conv_analysis_list[i]
        string = [gold_standard_analysis[0], gold_standard_analysis[1],
                  mystem_analysis[0], mystem_analysis[1],
                  natasha_analysis[0], natasha_analysis[1],
                  spacy_analysis[0], spacy_analysis[1]]
        tsv_output = csv.writer(file, delimiter='\t')
        tsv_output.writerow(string)

IndexError: list index out of range

4. (4 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker),  которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за  за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

## FAQ
1. В каком формате должна быть ручная разметка корпуса? - В любом, как вам удобно (табличка токен + тег, conllu, и тд). Не забудьте загрузить его на гитхаб вместе с решением.
2. Верно ли, что корпусом может быть набор не связанных между собой предложений? - Да.
3. Если случай спорный, у него может быть два разбора или нужно принять единое решение? - Может быть два разбора, если так подсказывает ваша лингвистическая интуиция.
4. Как оценивать accuracy, если POS-теггер не разрешает омонимию? - Проверить, что это точно так. За правильные ответы тогда можно считать случаи, когда тег из gold-разметки есть в ответе теггера, либо как это часто бывает, брать первый (нулевой) вариант - you decide.
5. Что делать со штуками вроде причастия? Если теггер указывает его как глагол, а мы разметили как причастие, это стоит засчитать (потому что у теггера не было возможности узнать наш тегсет) или нет? Или как раз на моменте выбора тегсета нужно продумать его так, чтобы по возможности избежать таких ситуаций?

\- Можно при конвертации тегов посмотреть, не нужно ли учитывать для этого теггера не только POS-тег, но и какие-то другие граммемы, чтобы потом преобразовать этот тег в причастие. Собственно, для таких случаев и нужна какая-то более хитрая конвертация, а не просто словарик соответствий POS-тегов."